In [ ]:
import base64
import os
from openai import OpenAI
import json
from skyview_dataset import skyview_data

api_key = ""
# Initialize your OpenAI client
client = OpenAI(api_key=api_key)

def encode_image(folder, image_filename):
    image_path = os.path.join("Aerial_Landscapes", folder, image_filename)
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')



# We define a system message to set the context
system_message = {
    "role": "system",
    "content": [
        {
        "type": "text",
        "text": """You are a helpful vision assistant that is going to help me understand the terrains of landscape images. The user wants to understand whether there is a place in the image that someone could land a small airplane. So being descriptive with the image given this context is crucial. 
            If there is no possible emergency landing areas then this is also something to consider. Keep your output to 5 sentences, with the last sentence giving a score on feasibility of landing an airplane. Specifically, the first four sentences will describe the landscape and the pros and cons of it given this emergency landing scenario. 
            The last sentence you should give a score out of 100 for whether the plane can land somewhere in the image or not, and the chance of survival if landed. 0 would indicate there is no survival chances, such as in mountainous areas or deep oceans and 100 would indicate an airport which is really safe to land on. 
            Note: a small airplane is 26 feet long with a wingspan of 36 feet and a cabin height of roughly 4 feet. Here is an example response: 
            1) The landscape shows a winding river surrounded by uneven terrain, with visible vegetation and some open areas.
            2) The riverbanks appear to have patches of flat land, but the irregularity and potential wetness of the soil pose risks.
            3) Vegetation is scattered, which could obstruct a small plane's landing, and the lack of a consistent open stretch limits options for a safe touchdown.
            4) While the river itself is unsuitable for landing, the narrow flat stretches near it may offer limited possibilities in an extreme emergency, though precision is critical.
            Score: 35/100 - The terrain presents significant challenges for landing, with survival chances depending on precision and the condition of the nearby flat areas."""
        }
    ]
}

for datum in skyview_data:
    base64_image = encode_image(datum["folder"], datum["image"])
    
    user_content = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        }
    ]
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            system_message,
            {
                "role": "user",
                "content": user_content
            }
        ],
    )
    
    answer = response.choices[0].message.content

    datum["gpt"] = answer

# Assuming 'data' is your updated list of dictionaries
with open("skyview_data_gpt.json", "w") as outfile:
    json.dump(skyview_data, outfile, indent=2)

# #TEST
# for d in skyview_data[:5]:  
#     print(f"ID: {d['id']}, Folder: {d['folder']}, Image: {d['image']}")
#     print("GPT Analysis:", d["gpt"])
#     print("---")
